# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [8]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('C:/Users/coolm/OneDrive - Vrije Universiteit Amsterdam/University documents/Third year/Text Mining/Assignments/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003', 'train.txt', ['words', 'pos', 'ignore', 'chunk'])
training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
    a_dict = {
      'words': token,
      'pos': pos
    }
    training_features.append(a_dict)
    training_gold_labels.append(ne_label)

   

In [9]:
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('C:/Users/coolm/OneDrive - Vrije Universiteit Amsterdam/University documents/Third year/Text Mining/Assignments/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

test_features = []
test_gold_labels = []
for token, pos, ne_label in train.iob_words():
    a_dict = {
        'words': token,
        'pos': pos
    }
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)

**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [16]:
from collections import Counter 

test = ConllCorpusReader('C:/Users/coolm/OneDrive - Vrije Universiteit Amsterdam/University documents/Third year/Text Mining/Assignments/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

for _, _, ne_label in train.iob_words():
    train_gold_labels.append(ne_label)

for _, _, ne_label in test.iob_words():
    test_gold_labels.append(ne_label)

train_instances = len(train_gold_labels)
test_instances = len(test_gold_labels)

train_label_counts = Counter(train_gold_labels)
test_label_counts = Counter(test_gold_labels)

print(f"Number of instances in training data: {train_instances}")
print(f"Number of instances in test data: {test_instances}")

print("Training data NERC label distribution:")
for label, count in train_label_counts.items():
    print(f"{label}: {count}")

print("Test data NERC label distribution:")
for label, count in test_label_counts.items():
    print(f"{label}: {count}")

print("Train vs Test:")
for label in set(train_label_counts.keys()).union(test_label_counts.keys()):
    train_count = train_label_counts.get(label, 0)
    test_count = test_label_counts.get(label, 0)
    print(f"{label}: Train = {train_count}, Test = {test_count}")



Number of instances in training data: 278610
Number of instances in test data: 278610
Training data NERC label distribution:
O: 229938
B-LOC: 10008
B-PER: 9702
I-PER: 6936
I-LOC: 1542
B-MISC: 4212
I-MISC: 1296
B-ORG: 9966
I-ORG: 5010
Test data NERC label distribution:
O: 229938
B-LOC: 10008
B-PER: 9702
I-PER: 6936
I-LOC: 1542
B-MISC: 4212
I-MISC: 1296
B-ORG: 9966
I-ORG: 5010
Train vs Test:
I-ORG: Train = 5010, Test = 5010
B-PER: Train = 9702, Test = 9702
B-MISC: Train = 4212, Test = 4212
B-ORG: Train = 9966, Test = 9966
O: Train = 229938, Test = 229938
I-MISC: Train = 1296, Test = 1296
I-LOC: Train = 1542, Test = 1542
B-LOC: Train = 10008, Test = 10008
I-PER: Train = 6936, Test = 6936


The training and test data are balanced in terms of label distribution so they have an equal number of NERC label instances.

The class '0' is the most frequent class making which causes an imbalance in the overall dataset and as a result can lead a bias towards predicting 0.

There aren't significant differences between the training and testing datasets so the model is unlikely to show any distribution shifts with the entity types.

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [17]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
vec = DictVectorizer()
the_array = vec.fit_transform(training_features + test_features)
train_instances = len(training_features)
X_train = the_array[:train_instances]
X_test = the_array[train_instances:]

y_train = train_gold_labels
y_test = test_gold_labels

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [8]:
from sklearn import svm

In [9]:
lin_clf = svm.LinearSVC()

In [3]:
##### [ YOUR CODE SHOULD GO HERE ]
# lin_clf.fit( # your code here

**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [10]:
# your code here

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [ ]:
import pandas

In [ ]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/Users/piek/Desktop/ONDERWIJS/data/nerc_datasets/kaggle/ner_v2.csv'
kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook